In [11]:

import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv('data/marketing_campaign.csv',header=0,sep='\t')

In [12]:
data

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [14]:

#Spending variable creation
data['Age']=2014-data['Year_Birth']

data['Spending']=data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
#Seniority variable creation
last_date = date(2014,10, 4)
data['Seniority']=pd.to_datetime(data['Dt_Customer'], dayfirst=True,format = '%d-%m-%Y')
data['Seniority'] = pd.to_numeric(data['Seniority'].dt.date.apply(lambda x: (last_date - x)).dt.days, downcast='integer')/30
data=data.rename(columns={'NumWebPurchases': "Web",'NumCatalogPurchases':'Catalog','NumStorePurchases':'Store'})
data['Marital_Status']=data['Marital_Status'].replace({'Divorced':'Alone','Single':'Alone','Married':'In couple','Together':'In couple','Absurd':'Alone','Widow':'Alone','YOLO':'Alone'})
data['Education']=data['Education'].replace({'Basic':'Undergraduate','2n Cycle':'Undergraduate','Graduation':'Postgraduate','Master':'Postgraduate','PhD':'Postgraduate'})

data['Children']=data['Kidhome']+data['Teenhome']
data['Has_child'] = np.where(data.Children> 0, 'Has child', 'No child')
data['Children'].replace({3: "3 children",2:'2 children',1:'1 child',0:"No child"},inplace=True)
data=data.rename(columns={'MntWines': "Wines",'MntFruits':'Fruits','MntMeatProducts':'Meat','MntFishProducts':'Fish','MntSweetProducts':'Sweets','MntGoldProds':'Gold'})


data=data[['Age','Education','Marital_Status','Income','Spending','Seniority','Has_child','Children','Wines','Fruits','Meat','Fish','Sweets','Gold']]
data.head()

,Age,Education,Marital_Status,Income,Spending,Seniority,Has_child,Children,Wines,Fruits,Meat,Fish,Sweets,Gold
0,57,Postgraduate,Alone,58138.0,1617,25.333333,No child,No child,635,88,546,172,88,88
1,60,Postgraduate,Alone,46344.0,27,7.000000,Has child,2 children,11,1,6,2,1,6
2,49,Postgraduate,In couple,71613.0,776,13.633333,No child,No child,426,49,127,111,21,42
3,30,Postgraduate,In couple,26646.0,53,7.866667,Has child,1 child,11,4,20,10,3,5
4,33,Postgraduate,In couple,58293.0,422,8.600000,Has child,1 child,173,43,118,46,27,15


In [15]:
data=data.dropna(subset=['Income'])
data=data[data['Income']<600000]

In [16]:

scaler=StandardScaler()
dataset_temp=data[['Income','Seniority','Spending']]
X_std=scaler.fit_transform(dataset_temp)
X = normalize(X_std,norm='l2')

gmm=GaussianMixture(n_components=4, covariance_type='spherical',max_iter=2000, random_state=5).fit(X)
labels = gmm.predict(X)
dataset_temp['Cluster'] = labels
dataset_temp=dataset_temp.replace({0:'Stars',1:'Need attention',2:'High potential',3:'Leaky bucket'})
data = data.merge(dataset_temp.Cluster, left_index=True, right_index=True)

pd.options.display.float_format = "{:.0f}".format
summary=data[['Income','Spending','Seniority','Cluster']]
summary.set_index("Cluster", inplace = True)
summary=summary.groupby('Cluster').describe().transpose()
summary.head()

Cluster       High potential  Leaky bucket  Need attention  Stars
Income count             462           641             528    584
       mean            73438         37705           69542  34757
       std             13753         12397           12006  12075
       min             49090          1730           44802   2447
       25%             65298         28839           60880  26489

In [17]:

PLOT = go.Figure()
for C in list(data.Cluster.unique()):
    

    PLOT.add_trace(go.Scatter3d(x = data[data.Cluster == C]['Income'],
                                y = data[data.Cluster == C]['Seniority'],
                                z = data[data.Cluster == C]['Spending'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}')

    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Income', titlefont_color = 'black'),
                                yaxis=dict(title = 'Seniority', titlefont_color = 'black'),
                                zaxis=dict(title = 'Spending', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}',
              'marker': {'line': {'width': 1}, 'size': 6},
              'mode': 'markers',
              'name': 'Need attention',
              'type': 'scatter3d',
              'x': array([58138., 55635., 82800., ..., 63777., 44802., 61223.]),
              'y': array([25.33333333, 23.        , 22.63333333, ..., 18.63333333, 25.8       ,
                          15.93333333]),
              'z': array([1617,  590, 1315, ...,  653, 1049, 1341])},
             {'hovertemplate': 'Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}',
              'marker': {'line': {'width': 1}, 'size': 6},
              'mode': 'markers',
              'name': 'Leaky bucket',
              'type': 'scatter3d',
              'x': array([46344., 26646., 58293., ..., 38054., 33562., 24434.]),
              'y': array([7.        , 7.86666667, 8.6       , ..., 5.2       , 3.36666667,
                          4.63333333]),
              'z': array([ 27,  53, 422, ...,  45,  51,  50])},
             {'hovertemplate': 'Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}',
              'marker': {'line': {'width': 1}, 'size': 6},
              'mode': 'markers',
              'name': 'High potential',
              'type': 'scatter3d',
              'x': array([71613., 62513., 63033., ..., 64014., 56981., 69245.]),
              'y': array([13.63333333, 13.        , 10.76666667, ...,  3.86666667,  8.4       ,
                           8.43333333]),
              'z': array([ 776,  716, 1102, ...,  444, 1241,  843])},
             {'hovertemplate': 'Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}',
              'marker': {'line': {'width': 1}, 'size': 6},
              'mode': 'markers',
              'name': 'Stars',
              'type': 'scatter3d',
              'x': array([33454., 30351.,  7500., ..., 26816., 34421., 52869.]),
              'y': array([17.13333333, 16.16666667, 23.        , ..., 25.93333333, 15.33333333,
                          23.96666667]),
              'z': array([169,  46,  61, ...,  22,  30, 172])}],
    'layout': {'autosize': True,
               'font': {'color': 'black', 'family': 'Gilroy', 'size': 12},
               'height': 800,
               'scene': {'xaxis': {'title': {'font': {'color': 'black'}, 'text': 'Income'}},
                         'yaxis': {'title': {'font': {'color': 'black'}, 'text': 'Seniority'}},
                         'zaxis': {'title': {'font': {'color': 'black'}, 'text': 'Spending'}}},
               'showlegend': True,
               'template': '...',
               'width': 800}
})

In [18]:
#Create Age segment
cut_labels_Age = ['Young', 'Adult', 'Mature', 'Senior']
cut_bins = [0, 30, 45, 65, 120]
data['Age_group'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = ['Low income', 'Low to medium income', 'Medium to high income', 'High income']
data['Income_group'] = pd.qcut(data['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = ['New customers', 'Discovering customers', 'Experienced customers', 'Old customers']
data['Seniority_group'] = pd.qcut(data['Seniority'], q=4, labels=cut_labels_Seniority)
data=data.drop(columns=['Age','Income','Seniority'])

In [19]:

cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
data['Wines_segment'] = pd.qcut(data['Wines'][data['Wines']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fruits_segment'] = pd.qcut(data['Fruits'][data['Fruits']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Meat_segment'] = pd.qcut(data['Meat'][data['Meat']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fish_segment'] = pd.qcut(data['Fish'][data['Fish']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Sweets_segment'] = pd.qcut(data['Sweets'][data['Sweets']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Gold_segment'] = pd.qcut(data['Gold'][data['Gold']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data.replace(np.nan, "Non consumer",inplace=True)
data.drop(columns=['Spending','Wines','Fruits','Meat','Fish','Sweets','Gold'],inplace=True)
data = data.astype(object)

In [21]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format
association=data.copy() 
df = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product='Wines'
segment='Biggest consumer'
target = '{\'%s_segment_%s\'}' %(product,segment)
results_personnal_care = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results_personnal_care

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4816,"(Cluster_Need attention, Income_group_High income)",(Wines_segment_Biggest consumer),0.121,0.249,0.084,0.697,2.800,0.054,2.476,0.731
4829,"(Cluster_Need attention, Seniority_group_Old customers)",(Wines_segment_Biggest consumer),0.123,0.249,0.085,0.691,2.779,0.054,2.433,0.730
9388,"(Education_Postgraduate, Meat_segment_Biggest consumer, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.122,0.249,0.081,0.664,2.670,0.051,2.237,0.713
1447,"(Education_Postgraduate, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.219,0.249,0.142,0.649,2.608,0.087,2.139,0.789
6115,"(Education_Postgraduate, Marital_Status_In couple, Cluster_Need attention)",(Wines_segment_Biggest consumer),0.139,0.249,0.090,0.648,2.606,0.055,2.135,0.715
4835,"(Cluster_Need attention, Meat_segment_Biggest consumer)",(Wines_segment_Biggest consumer),0.134,0.249,0.086,0.645,2.594,0.053,2.118,0.709
9765,"(Education_Postgraduate, Income_group_High income, Meat_segment_Biggest consumer)",(Wines_segment_Biggest consumer),0.177,0.249,0.112,0.634,2.550,0.068,2.054,0.738
338,(Cluster_Need attention),(Wines_segment_Biggest consumer),0.238,0.249,0.151,0.633,2.543,0.091,2.045,0.797
5071,"(Income_group_High income, Meat_segment_Biggest consumer)",(Wines_segment_Biggest consumer),0.190,0.249,0.119,0.626,2.517,0.072,2.010,0.744
6263,"(Education_Postgraduate, Income_group_High income, Marital_Status_In couple)",(Wines_segment_Biggest consumer),0.147,0.249,0.092,0.626,2.516,0.055,2.007,0.706
